In [635]:
from typing import List
import typing
import hashlib
import json
import pandas as pd
import random

# Generate Synthetic Data:

In [645]:
def random_id(length):
    number = '0123456789'
    alpha = 'abcdefghijklmnopqrstuvwxyz'
    id = ''
    for i in range(0,length):
        if(random.uniform(0, 1) > 0.5):
            id += random.choice(number)
        else:
            id += random.choice(alpha)
    return id

In [646]:
def generate_data(length):
    cust_data = {}
    for counter in range(0, length):
        cust_data.update({random_id(34): random.uniform(0.001, 10.000)})
    return(cust_data)

In [647]:
customer_data = generate_data(40)

with open("customer_data.json", "w") as fp:
    json.dump(customer_data, fp) 

In [649]:
f = open("customer_data.json")
customer_data = json.load(f)
f.close()

{'fu02l21z539axn7j550878h42oj0c0405d': 9.545324082105598, 'uk0mig2i8wr7sa2q6b22xjydv7ch65t0py': 6.174749396682197, '0w976djt5o185s1oc94xc43b0r3y1wgvmw': 5.96623345133311, 'x1w4nwf6lwksn5blp3u6h2753g25f013rv': 0.5989481490273911, 'u3u3l66n895h80mfe0u6jommxa534g71bm': 0.11482614087511749, '4ussfhgl5qmu76g4o040k9by45f966a92v': 2.8900076233297427, 'fo03fdw10nc4ktmkl571ltza5t5u6v593o': 7.494813582108022, 'menom1ip56fy054093qf4a5mlhm05seuk5': 4.600151530504669, 'u48t76v3rpys468yiydi82l9l8bh7j83mk': 6.325091975277764, 'k7s08lpky03u4661tg5sgn05o1roezbdd9': 1.9511832259388269, 'kolmim628li68ys37g4416q87wzie4yg4c': 1.1173699990736725, '3ci8c654nmjv32ya2j8sdjog2gh7u8g6kz': 2.723359254979569, '64604s948e37jl5n2y410j4ptgv4mc63x1': 6.834313748923529, 'l4lwy3yi3018b7pm1089hha38jj85d6zo2': 8.435472381300139, 'kb2854k2835595msa51m6oz80q34msnn90': 4.656812631224874, '694c6gs452w493i48dx4427319xy6t83y2': 9.277463214955347, 'v5xf79685lary4h7o80b8573t65k24lw4p': 4.185851014336689, 'c6gc8mnd388khid9trpr7k30

# Exchange Side:

In [595]:
class Node:
    def __init__(self, cust_id, node_id, left, right, addr: str, balance: float)-> None:
        self.node_id: Node = node_id
        self.cust_id: Node = cust_id
        self.left: Node = left
        self.right: Node = right
        self.addr = addr
        self.balance = balance

    @staticmethod
    def hash(val: str)-> str:
        return hashlib.sha256(val.encode('utf-8')).hexdigest()

    @staticmethod
    def doubleHash(val: str)-> str:
        return Node.hash(Node.hash(val))

In [608]:
class MerkleTree:
    def __init__(self, cust_data)-> None:
        addrs = list(cust_data.keys())
        balances = list(cust_data.values())
        while(math.log2(len(addrs)).is_integer() == False):
            addrs.append("")
            balances.append(0.0)
        self.__buildTree(addrs, balances)
        self.node_list = []
        self.tree_struct = []
    
    def __buildTree(self, addrs: List[str], balances: List[float])-> None:
        leaves: List[Node] = [Node(str(i), str(i), None, None, Node.doubleHash(addrs[i]), balances[i]) for i in range(0,len(addrs))]
        if len(leaves) % 2 == 1:
            leaves.append(leaves[-1:][0]) # duplicate last elem if odd number of elements
        self.root: Node = self.__buildTreeRec(leaves)

    def __buildTreeRec(self, nodes: List[Node])-> Node:
        half: int = len(nodes) // 2

        if len(nodes) == 2:
            return Node("#", str(nodes[0].node_id+nodes[1].node_id), nodes[0], nodes[1], Node.doubleHash(nodes[0].addr + nodes[1].addr), nodes[0].balance + nodes[1].balance)

        left: Node = self.__buildTreeRec(nodes[:half])
        right: Node = self.__buildTreeRec(nodes[half:])
        node_id: str = str(left.node_id + right.node_id) 
        addr: str = Node.doubleHash(left.addr + right.addr)
        balance: float = left.balance + right.balance
        return Node("#", node_id, left, right, addr, balance)
    
    def getTree(self):
        self.node_id = 0
        self.node_list = []
        self.tree_struct = []
        self.__getRecNodes(None, self.root)
        return([self.tree_struct, self.node_list])
    
    def __getRecNodes(self, node_id_parent, node):
        if node != None:
            self.node_list.append([node.node_id, str(node.addr), node.balance])
            node_id_left = self.__getRecNodes(node.node_id, node.left)
            node_id_right = self.__getRecNodes(node.node_id, node.right)
            self.tree_struct.append([node_id_left, node_id_right, node.node_id, node_id_parent, node.cust_id])
            return node.node_id
        else:
            return node_id_parent
    
    def printTree(self)-> None:
        self.__printTreeRec(self.root,"", "Root", 0)

    def __printTreeRec(self, node, space, child, dept)-> None:
        if node != None:
            print(space + node.cust_id+" "+str(node.node_id)+" "+child +" "+str(dept) + ": Node hash: " + str(node.addr)+": Node balance: "+str(round(node.balance,3)))
            
            self.__printTreeRec(node.left, space + "   ", "L", dept+1)
            self.__printTreeRec(node.right, space + "   ", "R", dept+1)

    def getRootHash(self)-> str:
        return [self.root.addr, self.root.balance]
    
    def saveTree(self)-> None:
        self.node_dict = self.getTree()
        with open("merkle_tree.json", "w") as fp:
            json.dump(self.node_dict , fp) 

In [650]:
mtree = MerkleTree(customer_data)
print(mtree.getRootHash())

['a7b9b07aecaaaa5129ca50b857c1f938684b26e63d6733dfa3a104616b8b2605', 202.00844948359045]


In [651]:
mtree.printTree()

# 0123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263 Root 0: Node hash: a7b9b07aecaaaa5129ca50b857c1f938684b26e63d6733dfa3a104616b8b2605: Node balance: 202.008
   # 012345678910111213141516171819202122232425262728293031 L 1: Node hash: 965dca0f07e5f65cbfb72592933210b7db06a18164a033fc88f3fa8b1b38c81e: Node balance: 162.954
      # 0123456789101112131415 L 2: Node hash: 9ad319d8fbbbf00725e6037912188e8c94a22202036337db0e0c874946b8b688: Node balance: 78.706
         # 01234567 L 3: Node hash: 955be65c02eb9f40c4596cef906ac815104a834df42c2aadefe51ffd2dfb544d: Node balance: 37.385
            # 0123 L 4: Node hash: 74cb5f988d5fd5b4745738951aaee1780d5a57a8715eae8a1428b83c81ddfc6e: Node balance: 22.285
               # 01 L 5: Node hash: a30d3cc621a0602c72a89caa91fd8271d514aa71dda2f9ee3827f5428b7721aa: Node balance: 15.72
                  0 0 L 6: Node hash: 0a49f53c70598b2bb77d55459886282acc42a48d2a29fe4e7cfac6622d27c8ea: Nod

In [652]:
mtree.getTree()

[[['0', '0', '0', '01', '0'],
  ['1', '1', '1', '01', '1'],
  ['0', '1', '01', '0123', '#'],
  ['2', '2', '2', '23', '2'],
  ['3', '3', '3', '23', '3'],
  ['2', '3', '23', '0123', '#'],
  ['01', '23', '0123', '01234567', '#'],
  ['4', '4', '4', '45', '4'],
  ['5', '5', '5', '45', '5'],
  ['4', '5', '45', '4567', '#'],
  ['6', '6', '6', '67', '6'],
  ['7', '7', '7', '67', '7'],
  ['6', '7', '67', '4567', '#'],
  ['45', '67', '4567', '01234567', '#'],
  ['0123', '4567', '01234567', '0123456789101112131415', '#'],
  ['8', '8', '8', '89', '8'],
  ['9', '9', '9', '89', '9'],
  ['8', '9', '89', '891011', '#'],
  ['10', '10', '10', '1011', '10'],
  ['11', '11', '11', '1011', '11'],
  ['10', '11', '1011', '891011', '#'],
  ['89', '1011', '891011', '89101112131415', '#'],
  ['12', '12', '12', '1213', '12'],
  ['13', '13', '13', '1213', '13'],
  ['12', '13', '1213', '12131415', '#'],
  ['14', '14', '14', '1415', '14'],
  ['15', '15', '15', '1415', '15'],
  ['14', '15', '1415', '12131415', '#'],


In [653]:
mtree.saveTree()

# Customer Side:

In [654]:
def doubleHash(val1, val2):
    if(val1 is None):
        val = hashlib.sha256(val2.encode('utf-8')).hexdigest()
        return hashlib.sha256(val.encode('utf-8')).hexdigest()
    elif(val2 is None):
        val = hashlib.sha256(val1.encode('utf-8')).hexdigest()
        return hashlib.sha256(val.encode('utf-8')).hexdigest()
    else:
        val = val1 + val2
        val = hashlib.sha256(val.encode('utf-8')).hexdigest()
        return hashlib.sha256(val.encode('utf-8')).hexdigest()

In [656]:
def validate_merkle_tree(customer_addr):
    root_hash = ""
    root_balance = 0.0
    
    # load customer data:
    f = open("customer_data.json")
    customer_data = json.load(f)
    f.close()
    
    # load merkle tree:
    f = open("merkle_tree.json")
    merkle_tree = json.load(f)
    f.close()
    df_tree_str = pd.DataFrame(data=merkle_tree[0],
                               columns=["Left Node ID",
                                        "Right Node ID",
                                        "Node ID",
                                        "Parent Node ID",
                                        "Customer ID"])
    node_dict = {}
    for item in merkle_tree[1]:
        node_dict.update({item[0]: [item[1], item[2]]})
    
    # get customer id:
    cust_id = None
    for index in range(0,len(customer_data)):
        if(list(customer_data.keys())[index] == customer_addr):
            cust_id = index
            break
    if cust_id is None:
        print("Unknown Customer!")
        return("")
    
    #first step is outside of the loop because leave1 depends on cust_id
    # get leaves:
    leave1 = df_tree_str[df_tree_str["Customer ID"]==str(cust_id)]
    parent = df_tree_str[df_tree_str["Node ID"]==leave1["Parent Node ID"].iloc[0]]
    leave2 = df_tree_str[df_tree_str["Parent Node ID"]==leave1["Parent Node ID"].iloc[0]]
    if(leave2.shape[0]==1):
        # this node is not a pair: parent has just one child!
        root_hash, root_balance = node_dict[leave1["Node ID"].iloc[0]]
    else:
        leave2 = leave2[leave2["Customer ID"] != str(cust_id)]
        # get hashes:
        hash1, balance1 = node_dict[leave1["Node ID"].iloc[0]]
        hash2, balance2 = node_dict[leave2["Node ID"].iloc[0]]

        root_balance = balance1 + balance2
        if(parent["Left Node ID"].iloc[0]==leave1["Node ID"].iloc[0]):
            root_hash = doubleHash(hash1, hash2)
        else:
            root_hash = doubleHash(hash2, hash1)
        
    parent_id = leave1["Parent Node ID"].iloc[0]
    # loop while parent_node_id != None:
    while(True):
        leave1 = df_tree_str[df_tree_str["Node ID"]==parent_id]
        parent = df_tree_str[df_tree_str["Node ID"]==leave1["Parent Node ID"].iloc[0]]
        leave2 = df_tree_str[df_tree_str["Parent Node ID"]==leave1["Parent Node ID"].iloc[0]]
        #if(leave2.shape[0]==1):
         # this node is not a pair: parent has just one child!
         #  leave2 = leave1
         #  root_balance = root_balance
         #  root_hash = root_hash
            
        if(leave2.shape[0]==2):
            leave2 = leave2[leave2["Node ID"] != leave1["Node ID"].iloc[0]]
            hash2, balance2 = node_dict[leave2["Node ID"].iloc[0]]
            root_balance = root_balance + balance2
            
            if(parent["Right Node ID"].iloc[0]==leave2["Node ID"].iloc[0]):
                root_hash = doubleHash(root_hash, hash2)
            else:
                root_hash = doubleHash(hash2, root_hash)
        
            parent_id = leave1["Parent Node ID"].iloc[0]
            if(df_tree_str[df_tree_str["Node ID"]==parent_id]["Parent Node ID"].iloc[0]==None):
                break
    
    return([root_hash, root_balance])

In [657]:
customer_addr = "v5xf79685lary4h7o80b8573t65k24lw4p"
validate_merkle_tree(customer_addr)

['a7b9b07aecaaaa5129ca50b857c1f938684b26e63d6733dfa3a104616b8b2605',
 202.00844948359045]

In [658]:
mtree.getRootHash()

['a7b9b07aecaaaa5129ca50b857c1f938684b26e63d6733dfa3a104616b8b2605',
 202.00844948359045]